# Cross-country analysis of the Covid-19 outbreak

Imports and set magics:

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets.widgets import interact, SelectMultiple

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import dataproject

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Read and clean data

We collect data from the "2019 Novel Coronavirus COVID-19 (2019-nCoV) Data Repository by Johns Hopkins CSSE", found on GitHub on the following link: https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series. The data is collected continuously and is thus updated whenever the file is run. However, we have commented on the data on the 29th of April so the comments do not necessarily match the shown data if a long time has passed since.
We use three datasets: The number of confirmed cases, the number of cases resulting in death and the number of cases recovered.

We begin by downloading a csv-file from the UN Population Prospects (https://population.un.org/wpp/Download/Standard/CSV/) from which we use the total population size (in thousands) in 2019 to calculate the number of Covid-19 cases per capita. 


In [3]:
# a. Load population data from UN
population = pd.read_csv('data/WPP2019_TotalPopulationBySex.csv')

# b. Select population in 2019 and only countries and population size
population = population.loc[population['Time']==2019]
PopTotal = population[['Location','PopTotal']].set_index('Location')
PopTotal.index.rename('Country/Region', inplace=True)

The country names included in the population data set do not exactly match the country names used in the Covid-19 data set. Therefore we change some of the names manually in the code below. The uncommented line in section a prints all country names in the population data set and has been used to find some of the countries, where the names do not match.

In [4]:
# a. Show country names included in the population data set
# print(PopTotal.index.values)

# b. Rename countries to match with Covid-19 data
PopTotal.rename({'United States of America': 'US', 
                 'Viet Nam': 'Vietnam',
                 'Iran (Islamic Republic of)': 'Iran'}, inplace = True) 

# c. Show the resulting dataframe
PopTotal.head()

,PopTotal
Country/Region,
Afghanistan,38041.757
Africa,1308064.176
African Group,1306320.572
African Union,1306903.030
African Union: Central Africa,154013.705


In [5]:
# a. load Covid-19 data
df_confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
df_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
df_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

# b. drop irrelevant columns
dfs = [df_confirmed, df_deaths, df_recovered]
drop_these = ['Lat', 'Long', 'Province/State']
for df in dfs:
    df.drop(drop_these, axis=1, inplace=True)
    
# c. group by country/region (we sum the number of cases across provinces/states)
df_confirmed = df_confirmed.groupby('Country/Region').sum()
df_deaths = df_deaths.groupby('Country/Region').sum()
df_recovered = df_recovered.groupby('Country/Region').sum()

The three Covid-19 datasets now look alike. The dataset of confirmed cases looks like this:

In [6]:
df_confirmed.head()

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,1279,1351,1463,1531,1703,1828,1939,2171,2335,2469
Albania,0,0,0,0,0,0,0,0,0,0,...,663,678,712,726,736,750,766,773,782,789
Algeria,0,0,0,0,0,0,0,0,0,0,...,3007,3127,3256,3382,3517,3649,3848,4006,4154,4295
Andorra,0,0,0,0,0,0,0,0,0,0,...,723,731,738,738,743,743,743,745,745,747
Angola,0,0,0,0,0,0,0,0,0,0,...,25,25,25,26,27,27,27,27,30,35


Now we create three new dataframes where we have the number of Covid-19 cases (confirmed, recovered and deaths) *per capita* (per thousand inhabitants) in each country. We first create a simple function that converts a dataframe to a per-capita dataframe, and then we call this function for each of the three dataframes.

In [7]:
def per_cap(df, pop = PopTotal):
    """ Convert dataframe to per capita numbers

    Args:
        df (pd.DataFrame): pandas dataframe 
        pop (pd.DataFrame): pandas dataframe with population for each country/region
        
    Returns:
        df_per_cap (pd.DataFrame): pandas dataframe converted to per capita numbers (per thousand inhabitants)

    """ 
    df_per_cap = pd.merge(df, pop, how = 'left', left_index=True, right_index=True)
    df_per_cap = df_per_cap.iloc[:,:-1].div(df_per_cap.iloc[:,-1], axis=0)
    return df_per_cap

In [8]:
# a. creating per-capita dataframes
df_confirmed_per_cap = per_cap(df_confirmed)
df_recovered_per_cap = per_cap(df_recovered)
df_deaths_per_cap = per_cap(df_deaths)

The per-capita dataframes now look like this:

In [9]:
df_confirmed_per_cap.head()

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.033621,0.035514,0.038458,0.040245,0.044767,0.048052,0.050970,0.057069,0.061380,0.064902
Albania,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.230135,0.235342,0.247144,0.252003,0.255475,0.260334,0.265888,0.268318,0.271442,0.273872
Algeria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.069844,0.072631,0.075628,0.078554,0.081690,0.084756,0.089378,0.093048,0.096486,0.099761
Andorra,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9.371840,9.475540,9.566277,9.566277,9.631089,9.631089,9.631089,9.657014,9.657014,9.682939
Angola,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000786,0.000786,0.000786,0.000817,0.000848,0.000848,0.000848,0.000848,0.000943,0.001100


**Convert to long format:** We convert the six datasets to long format in order to be able to make an interactive plot. The function `long()` is defined in the file `dataproject.py`.

In [10]:
deaths_long = dataproject.long(df_deaths, 'deaths')
recovered_long = dataproject.long(df_recovered, 'recovered')
confirmed_long = dataproject.long(df_confirmed, 'confirmed')

deaths_per_cap_long = dataproject.long(df_deaths_per_cap, 'deaths')
recovered_per_cap_long = dataproject.long(df_recovered_per_cap, 'recovered')
confirmed_per_cap_long = dataproject.long(df_confirmed_per_cap, 'confirmed')

# Explore data set

**General function to plot data**: We create a general function to plot the data, which we can call in the subsequent interactive plot

In [11]:
def plot_covid(per_cap, confirmed, deaths, recovered, confirmed_per_cap, deaths_per_cap, recovered_per_cap, dataset, country_region): 
    if per_cap == False:            # if per capita is not selected
        if dataset == 'Confirmed':  # if the dataset chosen is 'Confirmed'
            df = confirmed          # then choose the 'confirmed' dataframe
            y = 'confirmed'         # and use data from the column 'confirmed'
        elif dataset == 'Deaths':
            df = deaths
            y = 'deaths'
        else:
            df = recovered
            y = 'recovered'
            
    else:                           # if per capita is not selected
        if dataset == 'Confirmed':
            df = confirmed_per_cap
            y = 'confirmed'
        elif dataset == 'Deaths':
            df = deaths_per_cap
            y = 'deaths'
        else:
            df = recovered_per_cap
            y = 'recovered'
            
    I = df['Country/Region'] == country_region
    ax = df.loc[I,:].plot(x='date', y=y, style='-o')

**Interactive plot with all countries and regions**

In this plot, we have created drop-down menus, where you can select the dataset and country/region, you want to look at. By checking the box 'per thousand inhabitants', the graph is plotted per thousand inhabitants in the country chosen - else the absolute number is plotted. This setting can only be chosen for regular countries (i.e. not the Diamond Princess for example), because we only have population data for regular countries included in the UN data set. 

In [12]:
widgets.interact(plot_covid, 
    
    per_cap = widgets.Checkbox(value=False, description='per thousand inhabitants'),
    confirmed = widgets.fixed(confirmed_long),
    deaths = widgets.fixed(deaths_long),
    recovered = widgets.fixed(recovered_long),
    confirmed_per_cap = widgets.fixed(confirmed_per_cap_long),
    deaths_per_cap = widgets.fixed(deaths_per_cap_long),
    recovered_per_cap = widgets.fixed(recovered_per_cap_long),
                 
    dataset = widgets.Dropdown(description='Dataset', 
                               options=['Confirmed','Deaths','Recovered']),
    country_region = widgets.Dropdown(description='Country/Region', 
                                    options=confirmed_long['Country/Region'].unique())
                 
); 

interactive(children=(Checkbox(value=False, description='per thousand inhabitants'), Dropdown(description='Dat…

**Interactive plot with top countries**

For simplicity, we now construct a new, similar interactive plot, where we only show the top ten countries - as measured by the number of deaths.

In [13]:
# a. the total deaths in a country is given by the number of deaths at the latest date:
total_deaths = deaths_long.loc[deaths_long['date']==df_deaths.columns[-1]]

# b. the top 10 countries with the highest number of deaths
top_countries = total_deaths.sort_values(by = 'deaths', ascending=False).head(10)['Country/Region']

In [14]:
widgets.interact(plot_covid, 
    
    per_cap = widgets.Checkbox(value=False, description='per thousand inhabitants'),
    confirmed = widgets.fixed(confirmed_long),
    deaths = widgets.fixed(deaths_long),
    recovered = widgets.fixed(recovered_long),
    confirmed_per_cap = widgets.fixed(confirmed_per_cap_long),
    deaths_per_cap = widgets.fixed(deaths_per_cap_long),
    recovered_per_cap = widgets.fixed(recovered_per_cap_long),
                 
    dataset = widgets.Dropdown(description='Dataset', 
                               options=['Confirmed','Deaths','Recovered']),
    country_region = widgets.Dropdown(description='Country/Region', 
                                    options=top_countries)
                 
); 

interactive(children=(Checkbox(value=False, description='per thousand inhabitants'), Dropdown(description='Dat…

**Comment of the graph as of April 29th 2020**: From the above plot, we can see that China is affected first - the number of confirmed cases begins to increase around January 21st, and stabilizes around 80.000 in the beginning of March. 
To compare, Italy gets an approximately exponential growth in the number of confirmed cases from the beginning of March. Other European countries such as France, Spain and Belgium experience a similar exponential growth around the beginning to the mid of March -  a little later than in Italy.

# Analysis

Now we choose to create an interactive plot where we can see the development in the number of deaths for the top ten countries in the same graph. We start by creating the figure for total deaths and then we create the figure for deaths per thousand inhabitants. In the plot it is possible to select and deselect countries to show the development in multiple countries at the same time.

In [15]:
# a. selecting data for deaths for the top ten countries
df_deaths_t = df_deaths.transpose()
df_deaths_t = df_deaths_t[list(top_countries)]

# b. creating graph
sel_mul = SelectMultiple(description="Country/Region",options=df_deaths_t.columns,value=list(top_countries),disabled=False)
@interact(variables=sel_mul)
def plot_multiple(variables):
    df_deaths_t[list(variables)].plot()

interactive(children=(SelectMultiple(description='Country/Region', index=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9), optio…

In [16]:
# a. selecting data for deaths per capita for the top ten countries
df_deaths_per_cap_t = df_deaths_per_cap.transpose()
df_deaths_per_cap_t = df_deaths_per_cap_t[list(top_countries)]

# b. creating graph
sel_mul = SelectMultiple(description="Country/Region",options=df_deaths_per_cap_t.columns,value=list(top_countries),disabled=False)
@interact(variables=sel_mul)
def plot_multiple(variables):
    df_deaths_per_cap_t[list(variables)].plot()

interactive(children=(SelectMultiple(description='Country/Region', index=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9), optio…

From the above graphs we again see that China is the first country hit and is hit relatively hard quickly in absolute numbers. However as measured relative to the population size the story is different and the graph is almost flat compared to other countries.
We further see that the US is right now (as of April 29th) by far the worst hit in terms of absolute numbers (and has superseded Spain and Italy in a short time frame), whereas Belgium tops the list when measured per thousand inhabitants. In both graphs, we see that Italy is badly affected, both in terms of absolute deaths and deaths per thousand inhabitants.

We now create a table of the number of deaths in the top ten countries as well as the increase since the day before.

In [17]:
# a. We create a new dataframe, where we sort by date
# and calculate absolute and relative changes since the day before
df_long = deaths_long.sort_values(by = ['Country/Region', 'date'])
df_long['diff'] = df_long.groupby('Country/Region')['deaths'].diff()
df_long['diff_pct'] = df_long.groupby('Country/Region')['deaths'].apply(pd.Series.pct_change)*100
df_long = df_long.round(2)

# b. We find the total deaths as the number of deaths at the latest date
table = df_long.loc[df_long['date']==df_deaths.columns[-1]]
table.sort_values(by = 'deaths', ascending=False).head(10)

,Country/Region,date,variable,deaths,diff,diff_pct
19060,US,5/2/20,deaths,66369,1426.0,2.20
18972,Italy,5/2/20,deaths,28710,474.0,1.68
19064,United Kingdom,5/2/20,deaths,28205,622.0,2.26
19044,Spain,5/2/20,deaths,25100,557.0,2.27
18949,France,5/2/20,deaths,24763,166.0,0.67
18903,Belgium,5/2/20,deaths,7765,62.0,0.80
18953,Germany,5/2/20,deaths,6812,76.0,1.13
18910,Brazil,5/2/20,deaths,6761,349.0,5.44
18968,Iran,5/2/20,deaths,6156,65.0,1.07
19008,Netherlands,5/2/20,deaths,5003,94.0,1.91


**Comment as of April 29th 2020**: We see that the US has the highest number of deaths the 28th of April and has the highest absolute increase since the day before. China is number 10 on the list, without any increase in deaths. US, UK and Brazil are the countries that experience the highest absolute increase in the number of deaths since the day before. 

However, when we look at the percentage increase in the number of deaths, we find that Brazil is by far the country that experience the highest percentage growth. As epidemics are exponential by nature, the percentage increase per day is the most relevant measure if we want to predict, which  countries will suffer the most in the future.

# Conclusion

In this project, we have used data on the number of cases of Covid-19. We have constructed graphs depicting the development in the number of confirmed cases, the number of recovered cases and the number of deaths from the Covid-19 virus across countries. We have looked at the top ten countries as measured by the number of deaths and as of April 29th, the US is the fastest growing country in terms of absolute deaths *and* the country that has experienced most deaths in absolute terms. In terms of the number of deaths relative to the population size, Belgium is hit worst, followed by Spain and Italy In terms of relative increase in the number of deaths (which is the relevant measure for a pandemic, that is exponential by nature), Brazil tops the list. 